### Tuning Parameter

In [ ]:
# Define HOG parameters
orient = 9
pix_per_cell = 8
cell_per_block = 2

### Implementation

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.feature import hog
from skimage import color, exposure
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC

In [ ]:
cars = glob.glob('./DATASETS/vehicles_smallset/*/*.jpeg')
notcars = glob.glob('./DATASETS/non-vehicles_smallset/*/*.jpeg')

In [ ]:
# # Define a function to return HOG features and visualization
# def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis, feature_vec):
#         # Use skimage.hog() to get features only
#         features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell,pix_per_cell), 
#                                   cells_per_block=(cell_per_block, cell_per_block), visualise=vis, feature_vector=feature_vec)
#         return features

In [ ]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

In [ ]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

### Normalize Features


In [ ]:
features_list = []
for car in cars:
    img_bgr = cv2.imread(car)
    img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    features_hog = hog(gray, orientations=orient, pixels_per_cell=(pix_per_cell,pix_per_cell), 
                                  cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
    features_spatial = bin_spatial(img_bgr)
    features_color_hist = color_hist(img_bgr)
    features_list.append(np.concatenate((features_hog, features_spatial, features_color_hist)))
    
for notcar in notcars:
    gray = cv2.cvtColor(cv2.imread(notcar), cv2.COLOR_BGR2GRAY)
    features = hog(gray, orientations=orient, pixels_per_cell=(pix_per_cell,pix_per_cell), 
                                  cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
    features_spatial = bin_spatial(img_bgr)
    features_color_hist = color_hist(img_bgr)
    features_list.append(np.concatenate((features_hog, features_spatial, features_color_hist)))

features_array = np.asarray(features_list)
features_array.shape

In [ ]:
## why not vstack?
# y = np.vstack((np.ones((5,1)), np.zeros((4,1))))

In [ ]:
y = np.hstack((np.ones(len(cars)), 
              np.zeros(len(notcars))))

In [ ]:
print(y.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
# Fit a per-column scaler
X_scaler = StandardScaler().fit(features_array)
# Apply the scaler to X
scaled_X = X_scaler.transform(features_array)
scaled_X.shape

In [ ]:
plt.figure()
plt.boxplot(features_array, 1, '')

In [ ]:
plt.figure()
plt.boxplot(scaled_X, 1, '')

In [ ]:
# def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
#                         hist_bins=32, hist_range=(0, 256)):
#     # Create a list to append feature vectors to
#     features = []
#     # Iterate through the list of images
#     for file in imgs:
#         # Read in each one by one
#         image = mpimg.imread(file)
#         # apply color conversion if other than 'RGB'
#         if cspace != 'RGB':
#             if cspace == 'HSV':
#                 feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#             elif cspace == 'LUV':
#                 feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
#             elif cspace == 'HLS':
#                 feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
#             elif cspace == 'YUV':
#                 feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
#         else: feature_image = np.copy(image)      
#         # Apply bin_spatial() to get spatial color features
#         spatial_features = bin_spatial(feature_image, size=spatial_size)
#         # Apply color_hist() also with a color space option now
#         hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
#         # Append the new feature vector to the features list
#         features.append(np.concatenate((spatial_features, hist_features)))
#     # Return list of feature vectors
#     return features

In [ ]:
# car_features = extract_features(cars, cspace='HSV', spatial_size=(32, 32),
#                         hist_bins=32, hist_range=(0, 256))
# notcar_features = extract_features(notcars, cspace='HSV', spatial_size=(32, 32),
#                         hist_bins=32, hist_range=(0, 256))

In [ ]:
if len(features_array) > 0:
    car_ind = np.random.randint(0, len(cars))
    # Plot an example of raw and scaled features
    fig = plt.figure(figsize=(12,4))
    plt.subplot(131)
    plt.imshow(mpimg.imread(cars[car_ind]))
    plt.title('Original Image')
    plt.subplot(132)
    plt.plot(features_array[car_ind])
    plt.title('Raw Features')
    plt.subplot(133)
    plt.plot(scaled_X[car_ind])
    plt.title('Normalized Features')
    fig.tight_layout()
else: 
    print('Your function only returns empty feature vectors...')

### SVM or Decision Tree or Neural Networks

In [ ]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

In [ ]:
svc = LinearSVC()
svc.fit(X_train, y_train)

In [ ]:
print('Test Accuracy of SVC = ', svc.score(X_test, y_test))